<a href="https://colab.research.google.com/github/Maks2811/test_nagaev/blob/main/task_14_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [58]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col, sum, mean, month, year

spark = SparkSession.builder \
.appName("read_csv") \
.getOrCreate()

# создаем датафрейм из csv файла
df = spark.read.option("header", "true").csv("/content/weather_data.csv")
df.printSchema()

# преобразуем формат данных в колонке "date"
df = df.withColumn("date", to_date(df["date"], "yyyy-MM-dd"))

# преобразуем остальные колонки в тип данных с плавающей запятой
df = df.withColumn("temperature", col("temperature").cast("float"))
df = df.withColumn("precipitation", col("precipitation").cast("float"))
df = df.withColumn("wind_speed", col("wind_speed").cast("float"))

# Выводим схему после всех преобразований типов данных
df.printSchema()

df.show()

# проверяем наличие значений null в датафрейме
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

# 3.1 находим 5 самых жарких дней

hottest_days = df.select(col("date"), col("temperature"))

print("Результат 3.1")
hottest_days.orderBy(col("temperature").desc()).limit(5).show()


# 3.2 находим метеостанцию с наибольшим количеством осадков за последний год.

# выбираем данные только за последний год
last_year = df.filter(col("date") > "2022-12-31")
#last_year.show()

# вычисляем сумм осадков с группировкой по метеостанциям
agg_df = last_year.groupBy("station_id").agg(sum("precipitation"))

# делаем сортировку по убыванию и выводим станцию стоящую на 1 месте
print("Результат 3.2")
agg_df.orderBy(col("sum(precipitation)").desc()).limit(1).show()

# 3.3 Подсчитываем среднюю температуру по месяцам за все время наблюдений.

# добавляем новую колонку, которая содержжит толькол месяц из даты
month_temp = df.withColumn("month", month(col("date")))

# выбираем из датафрейма только 2 нужные колонки: месяц и температуру
month_temp = month_temp.select(col("month"), col("temperature"))

# группируем по месяцам, вычисляем среднее
agg_month_temp = month_temp.groupBy("month").agg(mean("temperature"))

print("Результат 3.3")
agg_month_temp.orderBy(col("month")).show()

spark.stop()

root
 |-- station_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- temperature: string (nullable = true)
 |-- precipitation: string (nullable = true)
 |-- wind_speed: string (nullable = true)

root
 |-- station_id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- temperature: float (nullable = true)
 |-- precipitation: float (nullable = true)
 |-- wind_speed: float (nullable = true)

+----------+----------+-----------+-------------+----------+
|station_id|      date|temperature|precipitation|wind_speed|
+----------+----------+-----------+-------------+----------+
| station_7|2022-06-28|  -6.751842|    23.670044|     5.459|
| station_4|2020-04-07|  -9.574844|    2.9858243| 6.8285055|
| station_8|2018-12-22|   19.34342|    33.582115|  8.975576|
| station_5|2021-09-09|  30.880953|    29.110437| 18.264654|
| station_7|2023-02-07|  23.459047|    49.510445| 3.7878683|
|station_10|2018-05-07|  27.525618|    11.957939|0.10053105|
| station_3|2019-08-27|  4.2